## The Movies Database

Given is the diagram of movies database consisting of 3 tables - Movies, Halls and Tickets

* Movie: This table has 4 columns - Movie_id (unique identifier for each Movie specific to language), Movie_name (Name of the movie), Language (Language of the Movie), Rating (Average rating given by viewers)
* Hall: This table has 3 columns - Hall_id (unique identifer for each Movie Hall), Hall_name (Name of the hall), Seating_capacity (maximum ticketed seats available in the hall)
* Ticket: This table has 3 columns - Movie_id (unique identifier for each Movie specific to language), Hall_id (unique identifer for each Movie Hall), Tickets_sold (number of tickets sold for the given Movie at the given Hall)

<img src="../../../images/movies_db.PNG" style="width: 65vw;"> <br>

<b>Tasks:</b>
1. Create an empty database named 'moviesdb' using sqlite terminal.
2. Create empty tables with connected relationships among the three tables as shown in above diagram. Using foreign key constraints and enforce referential integrity.
3. Extract the data from the three tables, links of which are provided in the code block. Load them into a dataframe and then into the table.
4. Write a query to extract data of which movie ran at over 80% of seating capacity and at which hall this was achieved. The query should extract all combinations of movie and hall with over 80% capacity.

In [1]:
movies_data_link = "../../../data/movie.csv"
halls_data_link = "../../../data/hall.csv"
tickets_data_link = "../../../data/ticket.csv"

In [2]:
!pip install tabulate

In [3]:
import sqlite3
import csv
import pandas as pd
from tabulate import tabulate

# Connecting to the database
vincon = sqlite3.connect('moviesDB.db')
vincon.execute("PRAGMA foreign_keys = ON")
vincur = vincon.cursor()


In [4]:
# creating movie table
try:
    vincur.execute("""CREATE TABLE movie(
    movie_id    INTEGER PRIMARY KEY, 
    movie_name  TEXT,
    language   TEXT,
    rating     INTEGER
    );""")
except Exception as e:
    print('Error:',e)


Error: table movie already exists


In [5]:
# creating movie hall
try:
    vincur.execute("""CREATE TABLE hall(
    hall_id    INTEGER PRIMARY KEY, 
    hall_name   TEXT,
    seating_capacity     INTEGER
    );""")
except Exception as e:
    print('Error:',e)


Error: table hall already exists


In [6]:
# create ticket table
try :
    vincur.execute('''CREATE TABLE ticket(
    movie_id     INTEGER ,
    hall_id      INTEGER ,
    tickets_sold  INTEGER    NON NULL,
    FOREIGN KEY(movie_id) REFERENCES movie(movie_id),
    FOREIGN KEY(hall_id) REFERENCES hall(hall_id)
    );''')
except Exception as e :
    print('Exception occurred :',e)

Exception occurred : table ticket already exists


In [7]:
# Reading data into tables
moviesdf = pd.read_csv(movies_data_link)
hallsdf = pd.read_csv(halls_data_link)
ticketsdf = pd.read_csv(tickets_data_link)

moviesdf['Movie_Id'] = [63, 56, 24, 63, 11]

try:
    moviesdf.to_sql(name='movie',con=vincon,if_exists='replace',index=False)
    hallsdf.to_sql(name='hall',con=vincon,if_exists='replace',index=False)
    ticketsdf.to_sql(name='ticket',con=vincon,if_exists='replace',index=False)
except Exception as e :
    print(e)

In [8]:
vincur.execute("""
SELECT * FROM movie
""")
vin_movie = vincur.fetchall()
headers_movie = ['Movie_Id', 'Movie_name', 'Language', 'Rating']
print(tabulate(vin_movie, headers_movie))

Movie_Id            Movie_name      Language
------------------  ------------  ----------
Lakewalker          Mandarin             6.4
Ramen loving Ronin  Japanese             8.1
Senjiruven          Tamil                7.3
The Unimaginable    English              7.9
La Belle            French               6.8
Kobali              Hindi                6.9
Kobali              Telugu               6.9


In [9]:
ticketsdf.head()

,Movie_id,Hall_id,Tickets_sold
0,63,14,100
1,56,2,106
2,24,13,34
3,63,1,136
4,11,15,114
